In [ ]:
# 주가 데이터를 이용하여 실습한다.
# 데이터는 일자, 시작가, 고가, 저가, 종가, 보정된 종가, 거래량으로 구성된 csv 파일이다.
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

df = pd.read_csv("../../../Pytorch_sample/data/kospi.csv")
scaler = MinMaxScaler()
df[["Open", "High", "Low", "Close", "Volume"]] = scaler.fit_transform(df[["Open", "High", "Low", "Close", "Volume"]])

In [ ]:
df.head()

In [ ]:
# 텐서 데이터 만들기
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
X = df[["Open", "High", "Low", "Volume"]].values
y = df["Close"].values
# 데이터프레임에서는 values를 이용하여 넘파이 배열로 만들 수 있다.

In [ ]:
def seq_data(x, y, sequence_length):
    x_seq = []
    y_seq = []
    for i in range(len(x) - sequence_length):
        x_seq.append(x[i:i+sequence_length]) # a[2:6] -> 2, 3, 4, 5
        y_seq.append(y[i+sequence_length])
    
    return torch.FloatTensor(x_seq).to(device), torch.FloatTensor(y_seq).to(device).view(-1, 1)
# view(-1, 1)을 사용하여 2차원으로 바꿔주는 이유는 MSE Loss가 기본적으로 2차원 타깃 데이터를 받기 때문이다.

In [ ]:
split = 200
sequence_length = 5 # sequence_length를 5로 설정하여 431-5 즉, 426개의 길이를 갖는 시퀀스 데이터를 만든다.
x_seq, y_seq = seq_data(X, y, sequence_length)
x_train_seq = x_seq[:split]
y_train_seq = y_seq[:split]
x_test_seq = x_seq[split:]
y_test_seq = y_seq[split:]
print(x_train_seq.size(), y_train_seq.size())
print(x_test_seq.size(), y_test_seq.size())

In [ ]:
train = torch.utils.data.TensorDataset(x_train_seq, y_train_seq)
test = torch.utils.data.TensorDataset(x_test_seq, y_test_seq)
batch_size = 20
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size)

In [ ]:
# RNN 구축에 필요한 하이퍼 파라미터 정의하기
input_size = x_seq.size(2) # 입력 변수의 개수
num_layers = 2 # 은닉 층의 개수
hidden_size = 8 # 은닉 상태를 저장하는 벡터의 크기

In [ ]:
# RNN 구축하기
class VanillaRNN(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
        super(VanillaRNN, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True)
        self.fc = nn.Sequential(nn.Linear(hidden_size*sequence_length, 1), nn.Sigmoid())

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device)
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [ ]:
# RNN 모델 불러오기
model = VanillaRNN(input_size=input_size,
                   hidden_size=hidden_size,
                   sequence_length=sequence_length,
                   num_layers=num_layers,
                   device=device
                   ).to(device)


In [ ]:
# 손실 함수 및 최적화 방법 정의
# 회귀문제이기에, MSE 손실 함수를 사용한다.
# 학습은 301회, 최적화 방법은 Adam을 사용한다.
criterion = nn.MSELoss()
num_epochs = 301
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# 모델 학습하기
loss_graph = []
n = len(train_loader)

for epoch in range(num_epochs):
    running_loss = 0.0

    for data in  train_loader:

        seq, target = data # 배치 데이터
        out = model(seq) # 출력값 산출
        loss = criterion(out, target) # 손실 함수 계산

        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # 최적화
        running_loss += loss.item()

    loss_graph.append(running_loss / n)
    if epoch % 100 == 0:
        print("[epoch:%d] loss:%.4f" %(epoch, running_loss/n))

In [ ]:
# 학습 손실 함수값 그리기
plt.figure(figsize=(20, 10))
plt.plot(loss_graph)
plt.show()

In [ ]:
# 주가 그리기
concatdata = torch.utils.data.ConcatDataset([train, test])
data_loader = torch.utils.data.DataLoader(dataset=concatdata, batch_size=100)
with torch.no_grad():
    pred = []
    model.eval()
    for data in data_loader:
        seq, target = data
        out = model(seq)
        pred += out.cpu().tolist()

plt.figure(figsize=(20, 10))
plt.plot(np.ones(100) * len(train), np.linspace(0, 1, 100), "--", linewidth = 0.6)
plt.plot(df["Close"][sequence_length:].values,"--")
plt.plot(pred, "b", linewidth = 0.6)
plt.legend(["train boundary", "actual", "prediction"])
plt.show()

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
        super(LSTM, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers=  num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_size*sequence_length, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [ ]:
# LSTM 모델 불러오기
model = LSTM(input_size=input_size,
                   hidden_size=hidden_size,
                   sequence_length=sequence_length,
                   num_layers=num_layers,
                   device=device
                   ).to(device)

In [ ]:
# 손실 함수 및 최적화 방법 정의
# 회귀문제이기에, MSE 손실 함수를 사용한다.
# 학습은 301회, 최적화 방법은 Adam을 사용한다.
criterion = nn.MSELoss()
num_epochs = 301
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# 모델 학습하기
loss_graph = []
n = len(train_loader)

for epoch in range(num_epochs):
    running_loss = 0.0

    for data in  train_loader:

        seq, target = data # 배치 데이터
        out = model(seq) # 출력값 산출
        loss = criterion(out, target) # 손실 함수 계산

        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # 최적화
        running_loss += loss.item()

    loss_graph.append(running_loss / n)
    if epoch % 100 == 0:
        print("[epoch:%d] loss:%.4f" %(epoch, running_loss/n))

In [ ]:
# 주가 그리기
concatdata = torch.utils.data.ConcatDataset([train, test])
data_loader = torch.utils.data.DataLoader(dataset=concatdata, batch_size=100)
with torch.no_grad():
    pred = []
    model.eval()
    for data in data_loader:
        seq, target = data
        out = model(seq)
        pred += out.cpu().tolist()

plt.figure(figsize=(20, 10))
plt.plot(np.ones(100) * len(train), np.linspace(0, 1, 100), "--", linewidth = 0.6)
plt.plot(df["Close"][sequence_length:].values,"--")
plt.plot(pred, "b", linewidth = 0.6)
plt.legend(["train boundary", "actual", "prediction"])
plt.show()

In [ ]:
# GRU 모델 구축
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
        super(GRU, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_size * sequence_length, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [ ]:
# GRU 모델 불러오기
model = GRU(input_size=input_size,
                   hidden_size=hidden_size,
                   sequence_length=sequence_length,
                   num_layers=num_layers,
                   device=device
                   ).to(device)

In [ ]:
# 손실 함수 및 최적화 방법 정의
# 회귀문제이기에, MSE 손실 함수를 사용한다.
# 학습은 301회, 최적화 방법은 Adam을 사용한다.
criterion = nn.MSELoss()
num_epochs = 301
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# 모델 학습하기
loss_graph = []
n = len(train_loader)

for epoch in range(num_epochs):
    running_loss = 0.0

    for data in  train_loader:

        seq, target = data # 배치 데이터
        out = model(seq) # 출력값 산출
        loss = criterion(out, target) # 손실 함수 계산

        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # 최적화
        running_loss += loss.item()

    loss_graph.append(running_loss / n)
    if epoch % 100 == 0:
        print("[epoch:%d] loss:%.4f" %(epoch, running_loss/n))

In [ ]:
# 주가 그리기
concatdata = torch.utils.data.ConcatDataset([train, test])
data_loader = torch.utils.data.DataLoader(dataset=concatdata, batch_size=100)
with torch.no_grad():
    pred = []
    model.eval()
    for data in data_loader:
        seq, target = data
        out = model(seq)
        pred += out.cpu().tolist()

plt.figure(figsize=(20, 10))
plt.plot(np.ones(100) * len(train), np.linspace(0, 1, 100), "--", linewidth = 0.6)
plt.plot(df["Close"][sequence_length:].values,"--")
plt.plot(pred, "b", linewidth = 0.6)
plt.legend(["train boundary", "actual", "prediction"])
plt.show()